In [1]:
import os
import pandas as pd
import pyspch
import pyspch.nn

D:\ProgramFiles\miniconda3\envs\pyspch_edit\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
# paths 

# public = /users/spraak/spchlab/public_html/pyspch/timit/ (final project)
# private = /esat/spchtemp/scratch/bvandyck/timit/ (dev project, training models on HTCondor)
remote_path = 'https://homes.esat.kuleuven.be/~spchlab/pyspch/timit/'

if True:
    # (@remote, personal machine, bvandyck) 
    timit_path = 'W:/timit/CDdata/timit/' # to extract corpus, features, labels
    write_path = 'Z:/scratch/bvandyck/timit/' # to write corpus, features, labels
    read_path = 'Z:/scratch/bvandyck/timit/' # to read corpus, features, labels

if False:
    # (@esat, bvandyck)
    timit_path = '/users/spraak/spchdata/timit/CDdata/timit/' # to extract corpus, features, labels
    write_path = '/esat/spchtemp/scratch/bvandyck/timit/' # to write corpus, features, labels
    read_path = '/esat/spchtemp/scratch/bvandyck/timit/' # to read corpus, features, labels
    
if False:
    # (@esat, spchlab)
    timit_path = '/users/spraak/spchdata/timit/CDdata/timit/' # to extract corpus, features, labels
    write_path = '/users/spraak/spchlab/public_html/pyspch/timit/' # to write corpus, features, labels
    read_path = '/users/spraak/spchlab/public_html/pyspch/timit/' # to read corpus, features, labels

os.chdir(write_path)

## Prepare (or read) TIMIT corpus 

In [3]:
# prepare TIMIT corpus 
prepare_corpus = True
read_corpus = True
write_corpus_path = write_path + 'data/dummy/'
read_corpus_path = read_path + 'data/dummy/'

if prepare_corpus:
    
    # get corpus from directory 
    timit_corpus = pyspch.timit.get_timit_corpus(timit_path) 
    timit_corpus = timit_corpus[:5]
    
    # write corpus to disk
    os.makedirs(write_corpus_path, exist_ok=True)
    pyspch.write_txt(timit_corpus, write_corpus_path + 'dummy.corpus')

    # extract meta data and write to disk
    timit_meta = pyspch.timit.get_timit_metadata(timit_corpus)
    timit_meta.to_csv(write_corpus_path + 'dummy.meta', sep='\t', index=False, header=False)

if read_corpus:
    
    # read corpus and meta data
    timit_corpus = pyspch.read_data_file(read_corpus_path + 'dummy.corpus')
    meta = pyspch.read_dataframe(read_corpus_path + "dummy.meta")
   
# print
print(f'Dummy corpus contains {len(timit_corpus)} files')  

Dummy corpus contains 5 files


## Read TIMIT data (wav)

In [4]:
# initialize SpchData with corpus
timit_data = pyspch.nn.SpchData(timit_corpus)

# read signals (wav-data) from disk ~ 25min
sample_rate_wav = 16000
timit_data.read_signals(timit_path, sample_rate_wav, extension='.wav')

## Extract TIMIT features (for exercise sessions)

### Mel filterbank cepstral coeffients (mfcc13)

In [5]:
# A. Mel Frequency Cepstral Coeffients (mfcc13)
write_feature_path = write_path + 'data/dummy/mfcc13/'

# arguments
feature_args = {
    'spg': None, 'Deltas': None, 'Norm': None,
    'sample_rate': 16000, 'f_shift': 0.01, 'f_length': 0.03,
    'preemp': 0.97, 'window': 'hamm', 'mode': 'dB',  
    'n_mels': 24, 'n_cep': 13 
    }

# extract and write features
pyspch.timit.make_dirs_for_corpus(write_feature_path, timit_corpus)
timit_data.extract_features_from_signals(feature_args)
timit_data.write_features(write_feature_path) # ~ 25min

# write feature_args 
feature_args_fname = os.path.join(write_feature_path, 'feature_args.json')
pyspch.write_json(feature_args, feature_args_fname)

In [6]:
# Feature extraction can also be done while reading the signals (wav-data).
# This requires less memory (since signals are not kept in memory).
# However, here we first load signals, then extract features, such that
# different feature extraction's can be performed, without re-reading the signals.
if False:
    # on the fly looks like:
    timit_data.extract_features(timit_path, feature_args, extension='.wav')

### Mel filterbanks (mel80)

In [7]:
# B. Mel filterbanks (mel80)
write_feature_path = write_path + 'data/dummy/mel80/'

# arguments
feature_args = {
    'spg': None, 'Deltas': None, 'Norm': None,
    'sample_rate': 16000, 'f_shift': 0.01, 'f_length': 0.03,
    'preemp': 0.97, 'window': 'hamm', 'mode': 'dB',
    'n_mels': 80, 'n_cep': None
    }

# extract and write features
pyspch.timit.make_dirs_for_corpus(write_feature_path, timit_corpus)
timit_data.extract_features_from_signals(feature_args)
timit_data.write_features(write_feature_path)

# write feature_args 
feature_args_fname = os.path.join(write_feature_path, 'feature_args.json')
pyspch.write_json(feature_args, feature_args_fname)

### Filterbanks (fb)

In [8]:
# C. Filterbanks (fb)
write_feature_path = write_path + 'data/dummy/fb/'

# arguments
feature_args = {
    'spg': None, 'Deltas': None, 'Norm': None,
    'sample_rate': 16000, 'f_shift': 0.01, 'f_length': 0.03,
    'preemp': 0.97, 'window': 'hamm', 'mode': 'dB',
    'n_mels': None, 'n_cep': None
    }

# extract and write features
pyspch.timit.make_dirs_for_corpus(write_feature_path, timit_corpus)
timit_data.extract_features_from_signals(feature_args)
timit_data.write_features(write_feature_path)

# write feature_args 
feature_args_fname = os.path.join(write_feature_path, 'feature_args.json')
pyspch.write_json(feature_args, feature_args_fname)

## Default setup for exercise sessions

Setup saved as pickled dataframe for fast loading:
- mfcc13 features:
- TIMIT61 phoneme labels: modified in exerice-session to 

Modified in exerice-session after reading to:
- mfcc39 features (by adding delta_ddelta and variance normalisation)
- TIMIT41 phoneme labels (by predefined mapping)

Setup split into train/test, smaller subsets can be defined analogously.

In [9]:
# Mel Frequency Cepstral Coeffients (mfcc13)
# instead of reading, extract features from signals (still in memory) ~ faster
read_feature_path = read_path + 'data/dummy/mfcc13/'
feature_args = pyspch.read_json(read_feature_path + 'feature_args.json')
timit_data.extract_features_from_signals(feature_args)

In [10]:
# TIMIT61 phoneme labels (phn)
read_label_path = read_path + 'data/segmentation/'
label_args = {'pad': 'h#', 'extension': '.phn'}
shift = feature_args['f_shift'] * feature_args['sample_rate']
timit_data.extract_alligned_labels(read_label_path, shift, label_args['pad'], label_args['extension'])

In [11]:
# split Spchdata into train/test
train_data = timit_data.subset_with_regex(f'.*(train)/.*')
test_data = timit_data.subset_with_regex(f'.*(test)/.*')

# to dataframe
train_df = train_data.to_dataframe()
test_df = test_data.to_dataframe()

# drop signals (wav-data)
train_df.drop(columns=['signals'], inplace=True)
test_df.drop(columns=['signals'], inplace=True)

In [12]:
# write setup to disk
write_setup_path = write_path + 'data/dummy/mfcc13/'
train_df.to_pickle(write_setup_path + 'train.pkl')
test_df.to_pickle(write_setup_path + 'test.pkl')

In [13]:
# read from disk
read_setup_path = write_path + 'data/dummy/mfcc13/'
train_df = pd.read_pickle(write_setup_path + 'train.pkl')
test_df = pd.read_pickle(write_setup_path + 'test.pkl')

In [16]:
# dataframe to SpchData
train_data = pyspch.nn.SpchData_from_dataframe(train_df)
test_data = pyspch.nn.SpchData_from_dataframe(test_df)

[array([[-8.89878540e+01, -8.85344086e+01, -8.88102875e+01, ...,
         -8.24185867e+01, -8.41160965e+01, -8.48701477e+01],
        [-3.24137259e+00, -3.68005610e+00, -2.64687681e+00, ...,
         -2.06441164e+00, -2.72634339e+00, -2.04581308e+00],
        [ 1.57882619e+00,  1.62022996e+00,  1.85508585e+00, ...,
         -1.05803418e+00, -7.73265362e-01, -3.21130246e-01],
        ...,
        [ 4.18425500e-01,  1.89767331e-01, -3.21906000e-01, ...,
         -8.03407073e-01,  1.38152689e-01, -3.14693093e-01],
        [-4.01160866e-02, -6.22037351e-02,  1.05652511e-01, ...,
          4.79777753e-01,  1.86404407e-01,  3.36148262e-01],
        [-2.02238187e-01, -1.24567240e-01,  1.42143413e-01, ...,
         -1.39512837e+00, -1.03967476e+00, -1.03731535e-01]], dtype=float32),
 array([[-9.0913391e+01, -9.1210777e+01, -9.1849274e+01, ...,
         -7.9598297e+01, -8.0518440e+01, -7.9906700e+01],
        [-4.8179998e+00, -4.9821968e+00, -4.5333366e+00, ...,
         -3.6911516e+00, -4.1874